In [51]:
import pandas as pd
import re

# prepare dataset
df=pd.read_csv('../rajoy.csv')
sentences=df["Text"].to_list()
# splits sentences using marks
split_stc=[re.split('(\.|\?|\!|;)',str(s)) for s in sentences]
stcs=[]
# joins with the mark again
for l in split_stc:
    liter=iter(l)
    stcs.append([h + next(liter,'') for h in liter])

# unflatten
sentences=[n.strip() for s in stcs for n in s if len(n.strip())>1]

In [52]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [70]:
from datasets import load_dataset

df=pd.DataFrame(sentences,columns=["text"])
df.to_json('../rajoy.json', orient="records", lines="true")
rajoy_ds=load_dataset('json',data_files='../rajoy.json', split=['train[10:20]'])


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/azureuser/.cache/huggingface/datasets/json/default-f6522c0fd47a2ec4/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 1)

Using custom data configuration default-f6522c0fd47a2ec4


In [67]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

rajoy_ds=rajoy_ds.map(tokenize_function,batched=True)

  0%|          | 0/57 [00:00<?, ?ba/s]

In [65]:
small_train_dataset = rajoy_ds["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = rajoy_ds["test"].shuffle(seed=42).select(range(1000))

KeyError: "Column train not in the dataset. Current columns in the dataset: ['text', 'input_ids', 'token_type_ids', 'attention_mask']"